# Deploy model API to Kubernetes

## Create a docker registry secret

This will allow kubernetes to pull images from docker hub (Public repository)

In [107]:
DOCKER_REGISTRY_SERVER='docker.io'
DOCKER_USER='xxxxxxxxx'
DOCKER_EMAIL='xxxxxxxxxxxxxxxxxxxxxxx'
DOCKER_PASSWORD='xxxxxxxxxxxxx'

!kubectl create secret docker-registry registry-credentials \
  --docker-server=$DOCKER_REGISTRY_SERVER \
  --docker-username=$DOCKER_USER \
  --docker-password=$DOCKER_PASSWORD \
  --docker-email=$DOCKER_EMAIL

secret/registry-credentials created


## Create deployments and pods

A Kubernetes deployment is a resource object in Kubernetes that provides declarative updates to applications. A deployment allows you to describe an application’s life cycle, such as which images to use for the app, the number of pods there should be, and the way in which they should be updated. 

A Kubernetes object is a way to tell the Kubernetes system how you want your cluster’s workload to look. After an object has been created, the cluster works to ensure that the object exists, maintaining the desired state of your Kubernetes cluster. 

A deployment ensures the desired number of pods are running and available at all times.

See more details @ https://www.redhat.com/en/topics/containers/what-is-kubernetes-deployment

In [1]:
!pygmentize ./kubernetes/kubernetes_API_deployment.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
    labels:
        app: placement-classifier-ml-api
    name: placement-classifier-ml
    namespace: fastapi-app-dev
spec:
    replicas: 1
    selector:
        matchLabels:
            app: placement-classifier
    template:
        metadata:
            labels:
                app: placement-classifier
        spec:
            containers:
            - image: kumarvc/fastapi-placement-classifier:1.0.2
              imagePullPolicy: IfNotPresent
              name: placement-classifier-container
              ports:
              - containerPort: 80
            imagePullSecrets:
            - name: registry-credentials


In [1]:
!kubectl create -f ./kubernetes/kubernetes_API_deployment.yaml

deployment.apps/placement-classifier-ml created


In [17]:
!kubectl delete pods placement-classifier-ml-7fb9fd7946-kzzbn -n fastapi-app-dev

pod "placement-classifier-ml-7fb9fd7946-kzzbn" deleted


In [31]:
!kubectl get pods -n fastapi-app-dev

NAME                                       READY   STATUS    RESTARTS   AGE
placement-classifier-ml-7fb9fd7946-fqlgv   1/1     Running   0          3m28s


In [30]:
!kubectl describe pods placement-classifier-ml-7fb9fd7946-fqlgv -n fastapi-app-dev

Name:         placement-classifier-ml-7fb9fd7946-fqlgv
Namespace:    fastapi-app-dev
Priority:     0
Node:         placement-classifier-worker/172.18.0.3
Start Time:   Wed, 06 Jan 2021 12:41:01 +0530
Labels:       app=placement-classifier
              pod-template-hash=7fb9fd7946
Annotations:  <none>
Status:       Running
IP:           10.244.2.2
IPs:
  IP:           10.244.2.2
Controlled By:  ReplicaSet/placement-classifier-ml-7fb9fd7946
Containers:
  placement-classifier-container:
    Container ID:   containerd://0844ec3a16f9be675e66cf3ab443b94ad1051b69413386442f0aeac377d0c354
    Image:          kumarvc/fastapi-placement-classifier:1.0.2
    Image ID:       docker.io/kumarvc/fastapi-placement-classifier@sha256:a613bc9dd0837701b059f9959d5404aed46948a1783097e0c1f0ea287ba628e2
    Port:           80/TCP
    Host Port:      0/TCP
    State:          Running
      Started:      Wed, 06 Jan 2021 12:44:06 +0530
    Ready:          True
    Restart Count:  0
    Environment:    <none>
   

## Create Service for accessing ML Service

A Kubernetes service is a logical abstraction for a deployed group of pods in a cluster. Since pods are ephemeral, a service enables a group of pods, which provide specific functions (REST prediction services) to be assigned a name and unique IP address.

For now, Service type has been defined as NodePort, so that you can access ML service with the help of node IP and nodeport (i.e defined in the kubernetes_API_service.yaml)

For real ML Service, service type must be defined as LoadBalancer so that request could be routed to any pods running in any of the available nodes

**Few importance concepts**
1) Port:  This expose kubernetes service on specified port within the cluster. Other pods within cluster can communicate with this on the specified port

2) TargetPort: This is the targetport on which service will send request to the running container inside the pod. Application running inside container will be listening on this port

3) NodePort: This is used to expose service externally to the cluster by the means of target nodeIP address. 

In [2]:
!pygmentize ./kubernetes/kubernetes_API_service.yaml

apiVersion: v1
kind: Service
metadata:
    labels:
        app: placement-classifier-svc
    name: placement-classifier-svc
    namespace: fastapi-app-dev
spec:
    ports:
    - name: predict
      nodePort: 30000
      port: 80
      targetPort: 80
    selector:
      app: placement-classifier
    type: NodePort


In [5]:
!kubectl create -f ./kubernetes/kubernetes_API_service.yaml

service/placement-classifier-svc created


In [6]:
!kubectl get svc

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   159m


## Send request to FastAPI model server running on Kubernetes

The Fast API server also provides a simple web UI dashboard (Swagger). Go to http://172.18.0.6:30000/docs in the browser and use the Web UI to send prediction request. 

**Note** Replace IP address from the kubernetes node ip (INTERNAL-IP). 

In [18]:
!kubectl get nodes -o wide

NAME                                 STATUS   ROLES    AGE   VERSION   INTERNAL-IP   EXTERNAL-IP   OS-IMAGE                                     KERNEL-VERSION     CONTAINER-RUNTIME
placement-classifier-control-plane   Ready    master   54m   v1.19.1   172.18.0.5    <none>        Ubuntu Groovy Gorilla (development branch)   5.4.0-59-generic   containerd://1.4.0
placement-classifier-worker          Ready    <none>   50m   v1.19.1   172.18.0.3    <none>        Ubuntu Groovy Gorilla (development branch)   5.4.0-59-generic   containerd://1.4.0
placement-classifier-worker2         Ready    <none>   50m   v1.19.1   172.18.0.7    <none>        Ubuntu Groovy Gorilla (development branch)   5.4.0-59-generic   containerd://1.4.0


## Check kubernetes logs to ensure ML API has started running

In [8]:
!kubectl logs placement-classifier-ml-7fb9fd7946-kzzbn -n fastapi-app-dev

Checking for script in /app/prestart.sh
Running script /app/prestart.sh
Running inside /app/prestart.sh, you could add migrations to this file, e.g.:

#! /usr/bin/env bash

# Let the DB start
sleep 10;
# Run migrations
alembic upgrade head

[2021-01-06 07:00:58 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2021-01-06 07:00:58 +0000] [1] [INFO] Listening at: http://0.0.0.0:80 (1)
[2021-01-06 07:00:58 +0000] [1] [INFO] Using worker: uvicorn.workers.UvicornWorker
[2021-01-06 07:00:58 +0000] [8] [INFO] Booting worker with pid: 8
[2021-01-06 07:00:58 +0000] [9] [INFO] Booting worker with pid: 9
[2021-01-06 07:00:58 +0000] [10] [INFO] Booting worker with pid: 10
[2021-01-06 07:01:21 +0000] [9] [INFO] Started server process [9]
[2021-01-06 07:01:21 +0000] [10] [INFO] Started server process [10]
[2021-01-06 07:01:21 +0000] [10] [INFO] Waiting for application startup.
[2021-01-06 07:01:21 +0000] [9] [INFO] Waiting for application startup.
[2021-01-06 07:01:21 +0000] [8] [INFO] Started server proc

### Send test request from Python program

In [9]:
import requests
data = {
  "sl_no": 112,
  "ssc_p": 84.0,
  "hsc_p": 90.9,
  "degree_p": 64.5,
  "etest_p": 86.04,
  "mba_p": 59.42,
  "gender": "M",
  "ssc_b": "Others",
  "hsc_b": "Others",
  "hsc_s": "Science",
  "degree_t": "Sci&Tech",
  "workex": "No",
  "specialisation": "Mkt&Fin"
}
response = requests.post("http://172.18.0.3:30000/predict", json=data)
print(response.text)

{"prediction":["Placed"]}


In [10]:
data = {
  "sl_no": 113,
  "ssc_p": 52.0,
  "hsc_p": 57.0,
  "degree_p": 50.8,
  "etest_p": 67.0,
  "mba_p": 62.79,
  "gender": "M",
  "ssc_b": "Central",
  "hsc_b": "Central",
  "hsc_s": "Commerce",
  "degree_t": "Comm&Mgmt",
  "workex": "No",
  "specialisation": "Mkt&HR"
}
response = requests.post("http://172.18.0.3:30000/predict", json=data)
print(response.text)

{"prediction":["Not Placed"]}


## Check kubernetes logs to ensure request has been received

In [14]:
!kubectl logs placement-classifier-ml-7fb9fd7946-kzzbn -n fastapi-app-dev

Checking for script in /app/prestart.sh
Running script /app/prestart.sh
Running inside /app/prestart.sh, you could add migrations to this file, e.g.:

#! /usr/bin/env bash

# Let the DB start
sleep 10;
# Run migrations
alembic upgrade head

[2021-01-06 07:00:58 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2021-01-06 07:00:58 +0000] [1] [INFO] Listening at: http://0.0.0.0:80 (1)
[2021-01-06 07:00:58 +0000] [1] [INFO] Using worker: uvicorn.workers.UvicornWorker
[2021-01-06 07:00:58 +0000] [8] [INFO] Booting worker with pid: 8
[2021-01-06 07:00:58 +0000] [9] [INFO] Booting worker with pid: 9
[2021-01-06 07:00:58 +0000] [10] [INFO] Booting worker with pid: 10
[2021-01-06 07:01:21 +0000] [9] [INFO] Started server process [9]
[2021-01-06 07:01:21 +0000] [10] [INFO] Started server process [10]
[2021-01-06 07:01:21 +0000] [10] [INFO] Waiting for application startup.
[2021-01-06 07:01:21 +0000] [9] [INFO] Waiting for application startup.
[2021-01-06 07:01:21 +0000] [8] [INFO] Started server proc